# Evaluation Pipeline Construction

## High Level Todos
- Establish guidelines
    - columns for attributes should be named `attribute_{NAME}`, if more than 2 groups in the attribute can be coded {0,1,2,...}
    - make one function that can process one csv file, evaluate_file
    - make another function that can do batch processing using the single file
  

In [49]:
# General Imports
import numpy as np
import pandas as pd
import os
import pathlib
import json

from fairlearn.metrics import (
    MetricFrame,
    selection_rate,
    false_positive_rate, # false positive error rate balance
    true_positive_rate, # false negative error rate balance
)

from sklearn.metrics import (
    accuracy_score, # use for both performance and fairness parts
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix
)

## Synthetic CSV Generation, for testing evaluation pipeline

input: file_name, num_rows
output: none

In [5]:
def generate_binary_prediction_csv(file_path, data_size = 1000):
    # Generate random data for y_true, y_pred, and attribute_gender
    y_true = np.random.randint(2, size=data_size)
    y_pred = np.random.randint(2, size=data_size)
    attribute_gender = np.random.randint(2, size=data_size) # assume binary attribute

    # Create a DataFrame using pandas
    df = pd.DataFrame({
        'y_true': y_true,
        'y_pred': y_pred,
        'attribute_gender': attribute_gender
    })

    # Write the DataFrame to a CSV file
    df.to_csv(file_path, index=False)

In [6]:
generate_binary_prediction_csv('synthetic_data.csv')

## Evaluation Metrics

Check what metrics only good for binary attributes

Aggregate Performance:
- accuracy
- f1-score
- precision
- recall

Fairness Metrics:
- Disparity
- predictive value parity
- Equalized Odds (Error Rate balance)
- Accuracy equality
- Treatment Equality


Further Ideas:
- should we process class probability labels

Functions:
- process file
    - input: file_name, assume it follows the formate assumptions, assume model_name is the file name before csv
    - output: returns a dict
- process folder (batch processing)

In [51]:
# Quick utility function to get confusion matrix
def _get_conf_mat_values(y_true, y_pred):
  cm = confusion_matrix(y_true, y_pred)

  # Extracting confusion matrix values
  TN = cm[0, 0]
  FP = cm[0, 1]
  FN = cm[1, 0]
  TP = cm[1, 1]

  return TN, FP, FN, TP

def predictive_parity(y_true, y_pred):
  TN, FP, FN, TP = _get_conf_mat_values(y_true, y_pred)
  return TP/(TP+FP)

def error_rate_ratio(y_true, y_pred):
  TN, FP, FN, TP = _get_conf_mat_values(y_true, y_pred)
  return FN/FP

def evaluate_file(path_obj):
    df = pd.read_csv(path_obj)

    model_name = path_obj.name

    y_true = df['y_true']
    y_pred = df['y_pred']
    sensitive_attribute = df['attribute_gender']

    # Fairness measurements processing
    metrics = {
              'selection_rate': selection_rate,
              'ppv': predictive_parity,
              'fp_err_rate_balance': false_positive_rate,
              'tp_error_rate_balance': true_positive_rate,
              'accuracy': accuracy_score,
              'error_rate_ratio': error_rate_ratio,
               }

    mf = MetricFrame(
                    metrics=metrics,
                    y_true=y_true,
                    y_pred=y_pred,
                    sensitive_features=sensitive_attribute
                    )

    results = {
        'model_performance': {'accuracy': accuracy_score(y_true, y_pred),
                              'f1_score': f1_score(y_true, y_pred),
                              'precision': precision_score(y_true, y_pred),
                              'recall': recall_score(y_true, y_pred),
                              },
        'fairness_performance': {
            'by_group_data': mf.by_group.to_dict(), # raw data
            'difference': mf.difference().to_dict(), # max inter-group diff per stat
            },
    }

    return results

def batch_evaluate(folder_path, write_name = None):
  """
  Processes model result CSV files in the

  input: folder_path: str
  returns: list of dicts containing performance results
  """
  folder = pathlib.Path(folder_path)

  perf_data = {}

  # Iterate through all the csv files in the folder
  for path in list(folder.glob('*.csv')):
    perf_data[str(path)] = evaluate_file(path)

  # Write data to 'write_name' json file
  if write_name is not None:
    with open(str(folder/write_name), "w") as outfile:
      json.dump(perf_data, outfile, indent=4)

  return perf_data

In [52]:
data = batch_evaluate('./model_data', 'perf_data.json')

In [53]:
pathlib.Path('./model_path').resolve()

PosixPath('/content/model_path')